## Prerequisites
This notebook requires an experimental version of the cmdty-storage package only published on Test PyPI, plus the packages curves for smoothing the forward curve, and ipysheet which provides the input grids.

`
pip install -i https://test.pypi.org/simple/ cmdty-storage==0.1.0a16
pip install curves
pip install ipysheet
`

If this notebook has been downloaded standalone, i.e. without downloading the whole storage repo,
it requires the file [storage_gui.py](https://github.com/cmdty/storage/blob/master/samples/python/storage_gui.py) be in the same directory.


In [1]:
%matplotlib inline
import storage_gui as sg
sg.display_gui()

### To Do
* Sort out truncation of labels.
* Interest rate curve, not flat rate.
* Ability to select granularity.
* Display more output data as tables using qgrid:
    * Trigger price.
    * Projected inventory.
    * Simulation results.
    

In [2]:
sg.test_data_btn()

Button(description='Populate Test Data', style=ButtonStyle())

In [3]:
def on_print_grid(b):
    display(sg.val_results_3f.deltas.to_frame())
    display(sg.val_results_3f.expected_profile)
    display(sg.val_results_3f.trigger_prices)

btn_print_grid = sg.ipw.Button(description='Display Data Grids')
btn_print_grid.on_click(on_print_grid)
btn_print_grid

Button(description='Display Data Grids', style=ButtonStyle())